In [87]:
import shutil
import os
import xml.etree.ElementTree as ET

import random
from glob import glob

import csv

### Преобразование разметочных данных XML в TXT

In [38]:
dataset_path = "data/LogoDet-3K"
output_img_dir = "data/preprocessed/images"
output_label_dir = "data/preprocessed/labels"

In [39]:
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

In [40]:
class_dict = {}
class_id_counter = 0

Переводим файлы XML в txt в новую папку preprocessed:

data/ \
├── preprocessed/ \
│   ├── images/ \
│   │   ├── category1/ \
│   │   ├── category2/ \
│   │   ├──  ... \
│   │   └── categoryn/ \
│   └── labels/ \
│       ├── category1/ \
│       ├── category2/ \
│       ├──  ... \
│       └── categoryn/ \
... \
Файлы сохраняем в формате brand_ph_number.txt и brand_ph_number.jpg

In [52]:
# Перебираем категории и бренды
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    
    for brand in os.listdir(category_path):
        # Если бренд ещё не добавлен в словарь, добавляем его с уникальным class_id
        if brand not in class_dict:
            class_dict[brand] = class_id_counter
            class_id_counter += 1

        brand_path = os.path.join(category_path, brand)
        
        # Перебираем все изображения и соответствующие XML файлы
        for filename in os.listdir(brand_path):
            if filename.endswith(".xml"):
                xml_path = os.path.join(brand_path, filename)
                image_path = os.path.join(brand_path, filename.replace(".xml", ".jpg"))
                
                try:
                    # Загружаем XML
                    tree = ET.parse(xml_path)
                    root = tree.getroot()

                    # Получаем размеры изображения
                    img_width = int(root.find("size/width").text)
                    img_height = int(root.find("size/height").text)
                    
                    # Получаем все объекты в изображении
                    labels = []
                    for obj in root.findall("object"):
                        class_name = obj.find("name").text.strip()  # Убираем лишние пробелы

                        # Проверка на наличие бренда в словаре
                        if class_name not in class_dict:
                            print(f"Бренд {class_name} не найден в словаре, добавляем его.")
                            class_dict[class_name] = class_id_counter
                            class_id_counter += 1
                        
                        xmin = int(obj.find("bndbox/xmin").text)
                        ymin = int(obj.find("bndbox/ymin").text)
                        xmax = int(obj.find("bndbox/xmax").text)
                        ymax = int(obj.find("bndbox/ymax").text)
                        
                        # Нормализуем координаты
                        x_center = (xmin + xmax) / 2 / img_width
                        y_center = (ymin + ymax) / 2 / img_height
                        width = (xmax - xmin) / img_width
                        height = (ymax - ymin) / img_height
                        
                        # Используем class_id из словаря
                        class_id = class_dict[class_name]

                        # Сохраняем в список
                        labels.append(f"{class_id} {x_center} {y_center} {width} {height}")
                    
                    # Генерация уникальных имен файлов
                    base_filename = f"{brand}_ph_{filename.split('.')[0]}"
                    unique_image_filename = f"{base_filename}.jpg"
                    unique_xml_filename = f"{base_filename}.xml"
                    unique_label_filename = f"{base_filename}.txt"

                    # Создаем подкаталоги для категории и бренда в папках images и labels
                    category_img_dir = os.path.join(output_img_dir, category)
                    category_label_dir = os.path.join(output_label_dir, category)

                    os.makedirs(category_img_dir, exist_ok=True)
                    os.makedirs(category_label_dir, exist_ok=True)

                    # Переносим изображение в папку images
                    dest_image_path = os.path.join(category_img_dir, unique_image_filename)
                    shutil.copy(image_path, dest_image_path)
                    
                    # Записываем метки в папку labels
                    unique_label_filepath = os.path.join(category_label_dir, unique_label_filename)
                    with open(unique_label_filepath, "w") as label_file:
                        label_file.write("\n".join(labels))  # Это записывание в файл внутри блока with
                except Exception as e:
                    print(f"Ошибка при обработке файла {xml_path}: {e}")

Бренд Alcatel-2 не найден в словаре, добавляем его.
Бренд alpinestars-2 не найден в словаре, добавляем его.
Бренд atari 2600-2 не найден в словаре, добавляем его.
Бренд Baume et Mercier-2 не найден в словаре, добавляем его.
Бренд Chipotle Mexican Grill-2 не найден в словаре, добавляем его.
Бренд Ck Calvin Klein-2 не найден в словаре, добавляем его.
Бренд ERKE-2 не найден в словаре, добавляем его.
Бренд Glock-2 не найден в словаре, добавляем его.
Бренд Harry Winston-2 не найден в словаре, добавляем его.
Бренд Harvest Moon не найден в словаре, добавляем его.
Бренд Heat-2 не найден в словаре, добавляем его.
Бренд Hummel-2 не найден в словаре, добавляем его.
Бренд Hurley-2 не найден в словаре, добавляем его.
Бренд Isabel Maran-2 не найден в словаре, добавляем его.
Бренд INOHERB-2 не найден в словаре, добавляем его.
Бренд jlindeberg-2 не найден в словаре, добавляем его.
Бренд Joma-2 не найден в словаре, добавляем его.
Бренд Karl Kani-2 не найден в словаре, добавляем его.
Бренд KATE SPADE-2 

In [53]:
class_dict['Alcatel-1']

615

### Разделение данных на train/test/val

Теперь нужно разделить набор данных на обучающие, тестовые и валидационные наборы. Они будут содержать 80%, 10% и 10% всех данных соответственно.

done/ \
├── images/ \
│   ├── train/  (80% тренировочных изображений) \
│   ├── val/    (10% валидационных изображений) \
│   ├── test/   (10% тестовых изображений) \
│ \
├── labels/ \
│   ├── train/  (аннотации для train) \
│   ├── val/    (аннотации для val) \
│   ├── test/   (аннотации для test) \
│ \
└── data.yaml   (файл конфигурации для YOLOv8) 

In [54]:
image_root = "data/preprocessed/images"
label_root = "data/preprocessed/labels"

In [55]:
all_images = glob(os.path.join(image_root, "*", "*.jpg"))
all_labels = glob(os.path.join(label_root, "*", "*.txt"))

In [56]:
image_dict = {os.path.splitext(os.path.basename(img))[0]: img for img in all_images}
label_dict = {os.path.splitext(os.path.basename(lbl))[0]: lbl for lbl in all_labels}

common_files = list(set(image_dict.keys()) & set(label_dict.keys()))

print(f"Изображений: {len(image_dict)}")
print(f"Разметок: {len(label_dict)}")
print(f"Пар: {len(common_files)}")

Изображений: 158654
Разметок: 158654
Пар: 158654


In [57]:
random.shuffle(common_files)

In [58]:
train_split = int(0.8 * len(common_files))
val_split = int(0.9 * len(common_files))

In [59]:
train_split

126923

In [60]:
train_files = common_files[:train_split]
val_files = common_files[train_split:val_split]
test_files = common_files[val_split:]

In [61]:
test_files[:10]

['rfeng_ph_77',
 'tafirol_ph_167',
 'zendium_ph_256',
 "rubio's fresh mexican grill_ph_80",
 'batman returns_ph_47',
 'zephyrhills_ph_59',
 'Alete_ph_38',
 'raisin bran crunch_ph_147',
 "Homer's Cinnamon Donut_ph_40",
 'Conoco_ph_96']

In [62]:
len(train_files)

126923

In [63]:
len(test_files)

15866

In [64]:
len(val_files)

15865

In [65]:
def move_files(file_list, split):
    for file in file_list:
        shutil.move(image_dict[file], os.path.join(f"data/done/images/{split}", f"{file}.jpg"))
        shutil.move(label_dict[file], os.path.join(f"data/done/labels/{split}", f"{file}.txt"))

In [66]:
move_files(train_files, "train")
move_files(val_files, "val")
move_files(test_files, "test")

### Замена & и ' в названиях файлов

Поскольку я собираюсь обучать модель в Kaggle, то для загрузки данных необходимо заменить некорректные символы.

In [84]:
folder_path = "data/done/images/val/"  # Укажите правильный путь к файлам

for root, dirs, files in os.walk(folder_path):
    for filename in files:
        new_filename = filename.replace("&", "_and_").replace("'", "_")  # Заменяем запрещённые символы
        if new_filename != filename:
            old_path = os.path.join(root, filename)
            new_path = os.path.join(root, new_filename)
            os.rename(old_path, new_path)
            print(f"Переименовано: {old_path} → {new_path}")

Переименовано: data/done/images/val/A. Favre & Fils_ph_32.jpg → data/done/images/val/A. Favre _and_ Fils_ph_32.jpg
Переименовано: data/done/images/val/Abercrombie & Fitch_ph_31.jpg → data/done/images/val/Abercrombie _and_ Fitch_ph_31.jpg
Переименовано: data/done/images/val/Abercrombie & Fitch_ph_33.jpg → data/done/images/val/Abercrombie _and_ Fitch_ph_33.jpg
Переименовано: data/done/images/val/Abercrombie & Fitch_ph_59.jpg → data/done/images/val/Abercrombie _and_ Fitch_ph_59.jpg
Переименовано: data/done/images/val/Abercrombie & Fitch_ph_64.jpg → data/done/images/val/Abercrombie _and_ Fitch_ph_64.jpg
Переименовано: data/done/images/val/Abercrombie & Fitch_ph_69.jpg → data/done/images/val/Abercrombie _and_ Fitch_ph_69.jpg
Переименовано: data/done/images/val/Adrian's_ph_17.jpg → data/done/images/val/Adrian_s_ph_17.jpg
Переименовано: data/done/images/val/Adrian's_ph_23.jpg → data/done/images/val/Adrian_s_ph_23.jpg
Переименовано: data/done/images/val/Adrian's_ph_38.jpg → data/done/images/va

Переименовано: data/done/images/val/Frosty O's_ph_23.jpg → data/done/images/val/Frosty O_s_ph_23.jpg
Переименовано: data/done/images/val/Frosty O's_ph_4.jpg → data/done/images/val/Frosty O_s_ph_4.jpg
Переименовано: data/done/images/val/Frosty O's_ph_5.jpg → data/done/images/val/Frosty O_s_ph_5.jpg
Переименовано: data/done/images/val/Gardetto's_ph_2.jpg → data/done/images/val/Gardetto_s_ph_2.jpg
Переименовано: data/done/images/val/Gardetto's_ph_21.jpg → data/done/images/val/Gardetto_s_ph_21.jpg
Переименовано: data/done/images/val/Gardetto's_ph_3.jpg → data/done/images/val/Gardetto_s_ph_3.jpg
Переименовано: data/done/images/val/Gardetto's_ph_35.jpg → data/done/images/val/Gardetto_s_ph_35.jpg
Переименовано: data/done/images/val/Gardetto's_ph_43.jpg → data/done/images/val/Gardetto_s_ph_43.jpg
Переименовано: data/done/images/val/Gardetto's_ph_59.jpg → data/done/images/val/Gardetto_s_ph_59.jpg
Переименовано: data/done/images/val/Gardetto's_ph_63.jpg → data/done/images/val/Gardetto_s_ph_63.jp

Переименовано: data/done/images/val/milo's hamburgers_ph_50.jpg → data/done/images/val/milo_s hamburgers_ph_50.jpg
Переименовано: data/done/images/val/milo's hamburgers_ph_61.jpg → data/done/images/val/milo_s hamburgers_ph_61.jpg
Переименовано: data/done/images/val/milo's hamburgers_ph_87.jpg → data/done/images/val/milo_s hamburgers_ph_87.jpg
Переименовано: data/done/images/val/milo's hamburgers_ph_91.jpg → data/done/images/val/milo_s hamburgers_ph_91.jpg
Переименовано: data/done/images/val/milo's hamburgers_ph_95.jpg → data/done/images/val/milo_s hamburgers_ph_95.jpg
Переименовано: data/done/images/val/moe's southwest grill_ph_10.jpg → data/done/images/val/moe_s southwest grill_ph_10.jpg
Переименовано: data/done/images/val/moe's southwest grill_ph_101.jpg → data/done/images/val/moe_s southwest grill_ph_101.jpg
Переименовано: data/done/images/val/moe's southwest grill_ph_11.jpg → data/done/images/val/moe_s southwest grill_ph_11.jpg
Переименовано: data/done/images/val/moe's southwest gr

### Создаем data.yaml

Для обучения YOLOv8 создаем data.yaml, где указываются пути к изображениям и количество классов.

In [70]:
brands = sorted(class_dict.keys())
len(brands)

3000

In [71]:
yaml_content = f"""
path: data/done
train: images/train
val: images/val
test: images/test 

nc: {len(brands)} 
names: {brands}
"""

with open("data/done/data.yaml", "w") as f:
    f.write(yaml_content.strip())

### Сохраняем словарь с индексами брендов

In [89]:
with open('class_dict.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(class_dict.keys())
    writer.writerow(class_dict.values())